In [15]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

IMG_SIZE = (128, 128)
BASE_DIR = "IRIS and FINGERPRINT DATASET"

def get_images_and_labels(base_dir):
    image_paths = []
    labels = []
    label_names = sorted([d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))])
    label_map = {name: idx for idx, name in enumerate(label_names)}
    
    for label in label_names:
        person_dir = os.path.join(base_dir, label)
        for subfolder in ['right', 'left', 'Fingerprint']:
            folder_path = os.path.join(person_dir, subfolder)
            if not os.path.exists(folder_path):
                continue
            for fname in os.listdir(folder_path):
                if fname.lower().endswith(('.bmp', '.jpg', '.jpeg', '.png')):
                    image_paths.append(os.path.join(folder_path, fname))
                    labels.append(label_map[label])
    return image_paths, labels, label_map

def preprocess_image(path):
    img = load_img(path, target_size=IMG_SIZE)
    img_array = img_to_array(img) / 255.0  # Normalize pixels to [0,1]
    return img_array

image_paths, labels, label_map = get_images_and_labels(BASE_DIR)
print(f"Found {len(image_paths)} images belonging to {len(label_map)} classes.")

# Preprocess images into arrays
image_arrays = np.array([preprocess_image(p) for p in image_paths])
labels_categorical = tf.keras.utils.to_categorical(labels, num_classes=len(label_map))


Found 900 images belonging to 45 classes.


In [16]:
BATCH_SIZE = 32

dataset = tf.data.Dataset.from_tensor_slices((image_arrays, labels_categorical))
dataset = dataset.shuffle(buffer_size=len(image_arrays))
dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(len(label_map), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(dataset, epochs=10)


Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 18s 420ms/step - accuracy: 0.0209 - loss: 3.8524
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 12s 420ms/step - accuracy: 0.0336 - loss: 3.8043
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 10s 344ms/step - accuracy: 0.0577 - loss: 3.7696
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 11s 390ms/step - accuracy: 0.0884 - loss: 3.5980
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 11s 369ms/step - accuracy: 0.1784 - loss: 3.1814
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 10s 326ms/step - accuracy: 0.2775 - loss: 2.6951
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 11s 379ms/step - accuracy: 0.4236 - loss: 2.0900
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 10s 349ms/step - accuracy: 0.5133 - loss: 1.8186
Epoch 9/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 10s 328ms/step - accuracy: 0.6415 - loss: 1.2714
Epoch 10/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 11s 367ms/step - accuracy: 0.7948 - loss: 0.7483


In [18]:
def predict_person(image_path, model, label_map):
    img = load_img(image_path, target_size=IMG_SIZE)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    predictions = model.predict(img_array)
    class_idx = np.argmax(predictions)
    inv_label_map = {v:k for k,v in label_map.items()}
    return inv_label_map[class_idx]

# Example
test_image = r"IRIS and FINGERPRINT DATASET\1\right\aevar5.bmp"
predicted_label = predict_person(test_image, model, label_map)
print(f"Predicted person ID: {predicted_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 325ms/step
Predicted person ID: 1
